# Case 2: Cargo Ship
The ship MS Leiden is traveling on a route involving five harbors in north-
ern Europe: Rotterdam, Hamburg, Kiel, Aarhus, and Copenhagen. You have
been asked to help the Captain with the loading and unloading plan (for the
destination harbors) that produces

- even and well-balanced solution
- a solution that is easy to unload (so that containers that are earlier un-loaded are not beneath containers that are later unloaded)
- solutions that can integrate as many containers as possible

The following details about the problem are provided:
1. The ship is now docked in Rotterdam. After visiting Rotterdam, the ship
will be traveling to Hamburg, Aarhus, and Copenhagen, in that order. The
containers destined for Rotterdam have been unloaded and the containers
destined for the remaining three harbors have to be loaded on the ship.
2. MS Leiden is a rather small container ship. It has eight bays, three tiers
and four rows. The layout of the ship is shown in the figure below.
3. Containers should be loaded such that a container that has to be unloaded
earlier should be placed in a higher position.
4. Each cell in the above figure is able to hold a forty-foot container. That
is, the ship has room for 8 × 3 × 4 = 96 forty-foot containers.
5. A forty-foot container can be placed on-top of another forty-foot container,
but not on top of an empty cell. We assume that only forty-foot containers
are loaded on the ship. Each container has a destination port.
6. Each container $i$ has a certain weight $w_i$. If a container is much heavier
than another $i$ container $j$, say $w_i − w_j > δ_w$, then it is not allowed to
place $w_i$ on top of $w_j$ .
7. The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [3]:
from desdeo_problem import variable_builder, ScalarObjective, ScalarConstraint, MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs import NSGAIII

import numpy as np

# desdeo.probdefs.Problem.__abstractmethods__ = set()
# desdeo docs: https://desdeo.readthedocs.io/en/latest/

![title](cargo1.png)

![title](cargo2.png)

![title](cargo3.png)

In [4]:
order = ['Rotterdam', 'Hamburg', 'Aarhus', 'Copenhagen']

## Custom dataset

- Load has to be balanced in every part of the journey (Rotterdam -> Hamburg, Hamburg -> Aarhus, Aarhus -> Copenhagen)
- 

In [5]:
# Different scenarios with different weights (ID, weight)
# The ID is the ID of the cargo and is between 1 and 10
# The weight is between 3750 and 27600 and random
HARBORS = ['Hamburg', 'Aarhus', 'Copenhagen']

# Scenario 1
Hamburg = 1
Aarhus = 2
Copenhagen = 3
min_weight = 1800 # in kg
max_weight = 28000 # in kg
Data = [(1, 5883), (1, 3485), (1, 9985), (1, 17600), 
        (2, 7352), (2, 9230), (2, 4699), (2, 16400), 
        (3, 14500), (3, 5773), (3, 12258)]

# Scenario 2 with different weights (ID, weight)


## Code

Balance score can be calculated as:
The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

### Helper functions

In [6]:
def remove_harbor_containers(configuration, destination):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
        destination: integer
            The destination harbor.
    Returns
    -------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    """
    tiers = range(configuration.shape[1])
    for tier in tiers:
        idx_container = np.argwhere(configuration[:, tier, :, 0] == destination)
        for idx in idx_container:
            if tier == 2:
                configuration[idx[0],tier,idx[1],:] = (0,0)
            # If the container is not on the top tier, move the container above it down
            else:
                configuration[idx[0],tier,idx[1],:] = configuration[idx[0],tier+1,idx[1],:]
                if tier == 1:
                    configuration[idx[0],tier+1,idx[1],:] = (0,0)
                else:
                    configuration[idx[0],tier+1,idx[1],:] = configuration[idx[0],tier+2,idx[1],:]
    return configuration

def positions_to_matrix(positions):

    configuration = np.zeros((8,3,4,2), dtype=int)

    for i, position in enumerate(positions):

        layer = int(position // 32)
        position = position % 32
        row = int(position // 4)
        column = int(position % 4)
        configuration[row, layer, column, :] = Data[i]
        
    return configuration

### Objective 1: Stability

In [7]:
def sum_tiers(configuration):

    only_weights = configuration[:, :, :, 1]
    summed_configuration = np.sum(only_weights, axis=1)

    return summed_configuration

def get_longitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=1)
    positions = np.arange(1, 9)

    center = np.sum(weights * positions) / np.sum(weights)

    return center

def get_lattitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=0)
    positions = np.arange(1, 5)

    center = np.sum(weights * positions) / np.sum(weights)

    return center

def calculate_centers(configuration):

    summed_configuration = sum_tiers(configuration)

    longitudinal_center = get_longitudinal_center(summed_configuration)
    lattitudinal_center = get_lattitudinal_center(summed_configuration)

    return longitudinal_center, lattitudinal_center

def get_score(configuration):

    longitudinal_center, lattitudinal_center = calculate_centers(configuration)

    long_error, latt_error = abs(longitudinal_center - 4.5), abs(lattitudinal_center - 2.5)

    return long_error + latt_error

def calculate_total_stability(candidate_solutions):

    stability_scores = []

    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)

        stability_score = 0

        for i in range(1, len(HARBORS)+1):

            stability_score += get_score(configuration)
            configuration = remove_harbor_containers(configuration, i)
        
        stability_scores.append(stability_score)
    
    return stability_scores

### Objective 2: Unloading time

In [81]:
def get_unloading_time(candidate_solutions):

    unloading_times = []

    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)
        only_harbors = configuration[:, :, :, 0]

        unloading_time = 0
        for i in range(8):
            for j in range(4):
                stack = only_harbors[i, :, j]
                stack = stack[stack != 0]
                if len(stack) > 1:
                    if tuple(stack) == (2,1,3):
                        return 4
                    for layer in range(len(stack)-1):
                        if stack[layer] == 1:
                            if stack[layer+1] != 1:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] != 1 and stack[layer+1] != 1:
                                    unloading_time += 2
                        elif stack[layer] == 2:
                            if stack[layer+1] == 3:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] == 3 and stack[layer+1] == 3:
                                    unloading_time += 2
        
        unloading_times.append(unloading_time)

    return unloading_times

In [82]:
def func(stack):
    unloading_time = 0
    if tuple(stack) == (2,1,3):
        return 4
    for layer in range(len(stack)-1):
        if stack[layer] == 1:
            if stack[layer+1] != 1:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] != 1 and stack[layer+1] != 1:
                    unloading_time += 2
        elif stack[layer] == 2:
            if stack[layer+1] == 3:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] == 3 and stack[layer+1] == 3:
                    unloading_time += 2
    return unloading_time

In [83]:
func([2, 1, 3])

4

In [84]:
# Constants
delta_weight = 500

# Constraints
# 1. Containers can't be placed on the same position
# 2. Containers can not levitate
# 3. Containers can only weigh delta_weight kg more than the container below it
# 
# Objectives
# 1. Minimize the long_error + latt_error (maximize the stability of the ship)
# 2. Minimize the unloading time of the ship

# Create Data
# n = np.random.randint(10, 96)
n = 55
h = np.random.randint(1, 4, n)
w = np.random.randint(2000, 30000, n)
Data = np.array(list(zip(h,w)))

# Create variables
var_names = ["c"+str(id) for id in range(len(Data))]

initial_values = [pos for pos in range(len(Data))]
lower_bounds = [0 for _ in range(len(Data))]
upper_bounds = [95.9999 for _ in range(len(Data))]

variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)

# Create objectives
objectives = [
    ScalarObjective(name="Stability", evaluator=calculate_total_stability, maximize=False),
    ScalarObjective(name="Unloading time", evaluator=get_unloading_time, maximize=False)
    ]

# Create constraints
def overlapping_eval(positions):

    positions = [int(pos) for pos in positions]

    if len(positions) != len(set(positions)):
        return -1
    else:
        return 1
    
def levitating_eval(positions):

    positions = [int(pos) for pos in positions]

    for pos in positions:
        if pos < 32:
            continue
        else:
            pos_below = np.arange(pos-32, -1, -32)
            if not all(x in positions for x in pos_below):
                return -1
    return 1

def weight_eval(positions):

    positions = [int(pos) for pos in positions]

    pos_weights = list(zip(positions, Data[:,1]))
    for pos, weight in pos_weights:
        if pos < 32:
            continue
        else:
            pos_below = pos-32
            # find weight of container below by getting the index in the list of tuples
            weight_below = pos_weights[positions.index(pos_below)][1]
            if weight > weight_below + delta_weight:
                return -1
    return 1


constraints = [
    ScalarConstraint(name="Overlapping", n_decision_vars=len(Data), n_objective_funs=0, evaluator=overlapping_eval),
    ScalarConstraint(name="Levitating", n_decision_vars=len(Data), n_objective_funs=0, evaluator=levitating_eval),
    ScalarConstraint(name="Weight", n_decision_vars=len(Data), n_objective_funs=0, evaluator=weight_eval)
    ]

problem = MOProblem(objectives, variables, constraints)

# Create an algorithm
evolver = NSGAIII(problem, 
                  n_iterations=10,
                  n_gen_per_iter=100,
                  population_size=100)

# Run the algorithm
while evolver.continue_evolution():
    evolver.iterate()

ConstraintError: Objective decision_vector [[0.28881814 4.        ]
 [1.63786123 4.        ]
 [2.67376514 4.        ]
 [2.43601108 4.        ]
 [2.83858574 4.        ]
 [0.98773342 4.        ]
 [2.90868898 4.        ]
 [1.25657024 4.        ]
 [0.77144268 4.        ]
 [2.4194259  4.        ]
 [2.11853904 4.        ]
 [2.03002207 4.        ]
 [0.73600059 4.        ]
 [1.08740742 4.        ]
 [2.18416909 4.        ]
 [1.52954439 4.        ]
 [1.3252079  4.        ]
 [3.08883566 4.        ]
 [2.43680167 4.        ]
 [0.69704734 4.        ]
 [0.61581277 4.        ]
 [1.58448731 4.        ]
 [1.07735524 4.        ]
 [2.76984237 4.        ]
 [2.81884531 4.        ]
 [1.35863849 4.        ]
 [0.62915233 4.        ]
 [0.63760224 4.        ]
 [0.98693318 4.        ]
 [0.61853715 4.        ]
 [0.47987062 4.        ]
 [1.15736633 4.        ]
 [2.09804208 4.        ]
 [0.98949998 4.        ]
 [1.16842922 4.        ]
 [1.18207621 4.        ]
 [1.76265705 4.        ]
 [0.72420644 4.        ]
 [0.939